<a href="https://colab.research.google.com/github/JasonD28/CS-175/blob/master/model/faster_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install pydicom

     |████████████████████████████████| 35.5MB 83kB/s 


In [3]:
%%shell
git clone https://github.com/cocodataset/cocoapi.git
cd cocoapi
cp PythonAPI/pycocotools /usr/local/lib/python3.6/dist-packages/ -r

Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 29.28 MiB/s, done.
Resolving deltas: 100% (576/576), done.


In [4]:
%%shell
git clone https://github.com/JasonD28/CS-175.git
cd CS-175
cp model/dataset.py  ../

Cloning into 'CS-175'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 180 (delta 1), reused 0 (delta 0), pack-reused 172
Receiving objects: 100% (180/180), 1014.00 KiB | 8.05 MiB/s, done.
Resolving deltas: 100% (91/91), done.


In [5]:
%%shell

# Using PyTorch helper functions to simplify training
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.5.1

cp references/detection/utils.py ../
cp references/detection/engine.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/coco_utils.py ../

Cloning into 'vision'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 8390 (delta 0), reused 2 (delta 0), pack-reused 8384
Receiving objects: 100% (8390/8390), 10.25 MiB | 21.15 MiB/s, done.
Resolving deltas: 100% (5774/5774), done.
Branch 'v0.5.1' set up to track remote branch 'v0.5.1' from 'origin'.
Switched to a new branch 'v0.5.1'


In [0]:
import torch
import utils
from torch import optim
import torchvision
from dataset import PneumoniaDataset
from engine import train_one_epoch, evaluate
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

root = "drive/My Drive/cs-175-project/cs-175-data"

In [0]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=3)

In [0]:
# model.load_state_dict(torch.load('drive/My Drive/cs-175-project/model_checkpoints/sixth_try_model.pt'))

In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
dataset = PneumoniaDataset(root, True)
test_dataset = PneumoniaDataset(root, True)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-1000])
test_dataset = torch.utils.data.Subset(test_dataset, indices[-1000:])

data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=20, shuffle=True, num_workers=8,
        collate_fn=utils.collate_fn)

test_data_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=5e-4)
lr_step = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=5,
                                               gamma=0.1)


In [5]:
for epoch in range(7):
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=20)
    lr_step.step()
    torch.save(model.state_dict(), 'drive/My Drive/cs-175-project/model_checkpoints/eighth_try_model.pt')
    evaluate(model, test_data_loader, device=device)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


Epoch: [0]  [   0/1285]  eta: 2:13:52  lr: 0.000001  loss: 3.0121 (3.0121)  loss_classifier: 1.0658 (1.0658)  loss_box_reg: 0.0129 (0.0129)  loss_objectness: 1.3827 (1.3827)  loss_rpn_box_reg: 0.5507 (0.5507)  time: 6.2510  data: 3.7788  max mem: 12644
Epoch: [0]  [  20/1285]  eta: 0:52:44  lr: 0.000011  loss: 1.9558 (2.0341)  loss_classifier: 0.4940 (0.5835)  loss_box_reg: 0.0090 (0.0105)  loss_objectness: 0.9641 (0.9217)  loss_rpn_box_reg: 0.5282 (0.5184)  time: 2.3137  data: 0.0549  max mem: 13061
Epoch: [0]  [  40/1285]  eta: 0:49:46  lr: 0.000021  loss: 0.3247 (1.2318)  loss_classifier: 0.0523 (0.3327)  loss_box_reg: 0.0098 (0.0115)  loss_objectness: 0.0387 (0.4943)  loss_rpn_box_reg: 0.2111 (0.3932)  time: 2.2912  data: 0.0467  max mem: 13061
Epoch: [0]  [  60/1285]  eta: 0:48:17  lr: 0.000031  loss: 0.1762 (0.8883)  loss_classifier: 0.0446 (0.2384)  loss_box_reg: 0.0140 (0.0134)  loss_objectness: 0.0128 (0.3369)  loss_rpn_box_reg: 0.1018 (0.2996)  time: 2.2959  data: 0.0522  max

In [0]:
model2 = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features2 = model2.roi_heads.box_predictor.cls_score.in_features
model2.roi_heads.box_predictor = FastRCNNPredictor(in_features2, num_classes=3)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model2.to(device)
dataset2 = PneumoniaDataset(root, True)
test_dataset2 = PneumoniaDataset(root, True)
indices = torch.randperm(len(dataset2)).tolist()
dataset2 = torch.utils.data.Subset(dataset2, indices[:-50])
test_dataset2 = torch.utils.data.Subset(test_dataset2, indices[-50:])

data_loader2 = torch.utils.data.DataLoader(
        dataset2, batch_size=10, shuffle=True, num_workers=6,
        collate_fn=utils.collate_fn)

test_data_loader2 = torch.utils.data.DataLoader(
    test_dataset2, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)
params = [p for p in model2.parameters() if p.requires_grad]
optimizer2 = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
lr_step2 = torch.optim.lr_scheduler.StepLR(optimizer2,
                                               step_size=5,
                                               gamma=0.1)
for epoch in range(2):
    train_one_epoch(model2, optimizer2, data_loader2, device, epoch, print_freq=40)
    lr_step2.step()
    torch.save(model2.state_dict(), 'drive/My Drive/cs-175-project/model_checkpoints/second_try_model.pt')
    evaluate(model2, test_data_loader2, device=device)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


Epoch: [0]  [   0/2664]  eta: 5:33:19  lr: 0.000010  loss: 1.9302 (1.9302)  loss_classifier: 0.4352 (0.4352)  loss_box_reg: 0.0099 (0.0099)  loss_objectness: 1.0406 (1.0406)  loss_rpn_box_reg: 0.4444 (0.4444)  time: 7.5074  data: 5.4561  max mem: 7168
Epoch: [0]  [  40/2664]  eta: 1:32:11  lr: 0.000210  loss: 0.3582 (1.0259)  loss_classifier: 0.0314 (0.1667)  loss_box_reg: 0.0024 (0.0052)  loss_objectness: 0.0732 (0.4746)  loss_rpn_box_reg: 0.2473 (0.3794)  time: 1.9481  data: 0.0343  max mem: 7434
Epoch: [0]  [  80/2664]  eta: 1:27:39  lr: 0.000410  loss: 0.1192 (0.5948)  loss_classifier: 0.0189 (0.0933)  loss_box_reg: 0.0057 (0.0057)  loss_objectness: 0.0177 (0.2568)  loss_rpn_box_reg: 0.0740 (0.2390)  time: 1.9622  data: 0.0323  max mem: 7434
Epoch: [0]  [ 120/2664]  eta: 1:25:13  lr: 0.000609  loss: 0.0866 (0.4308)  loss_classifier: 0.0256 (0.0702)  loss_box_reg: 0.0098 (0.0069)  loss_objectness: 0.0157 (0.1786)  loss_rpn_box_reg: 0.0358 (0.1751)  time: 1.9600  data: 0.0336  max me

Process Process-12:
Process Process-11:
Process Process-13:
Process Process-14:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/util.py", line 322, in _exit_function
    _run_finalizers()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/usr/lib/python3.6/multiprocessing/util.py", line 322, in _exit_function
    _run_finalizers()
  File "/usr/lib/python3.6/multiprocessing/util.py", line 322, in _exit_function
    _run_finalizers()
  File "/usr/lib/python3.6/multiprocessing/util

KeyboardInterrupt: ignored